In [ ]:
from typing import List, Tuple
from pyhepmc import GenEvent, GenParticle, ReaderAsciiHepMC2, FourVector
import numpy as np

MOMENTUM_ORDER = True

def calculate_trans_momentum(electron):
    """Function to calculate transverse momentum of a GenParticle Object"""
    px = electron.momentum.px
    py = electron.momentum.py
    return np.sqrt(px**2 + py**2)

def get_z_decay_electrons(event):
    electrons = []
    for p in event.particles:
        if p.pid == 23:
            vz = p.end_vertex
            if vz is None:
                continue
            for dau in vz.particles_out:
                if dau.status == 1 and abs(dau.pid) == 11:
                    electrons.append(dau)
    return electrons


def extract_electron_momenta(event):
    electrons = get_z_decay_electrons(event)
    if len(electrons) < 2:
        raise ValueError(f"Expected at least 2 final-state electrons/positrons, found {len(electrons)}")
    e1, e2 = electrons[0], electrons[1]
    elpt, e2pt = calculate_trans_momentum(e1), calculate_trans_momentum(e2)
    if not MOMENTUM_ORDER:
        return (e1.momentum, e2.momentum)
    if elpt >= e2pt:
        return (e1.momentum, e2.momentum)
    else:
        return (e2.momentum, e1.momentum)


def extract_electron_momenta_from_hepmc2(input_path):
    momenta_list = []
    total = 0
    skipped = 0
    for paths in input_path:
        with ReaderAsciiHepMC2(paths) as reader:
            while not reader.failed():
                evt = GenEvent()
                reader.read_event(evt)
                if reader.failed():
                    break

                total += 1
                try:
                    mom_pair = extract_electron_momenta(evt)
                    momenta_list.append(mom_pair)
                except ValueError:
                    skipped += 1
    print(f"Processed {total} events, extracted from {len(momenta_list)} events, skipped {skipped} events.")
    return momenta_list


import numpy as np

def write_momenta_to_npz(momenta, output_path):
    # Define column names
    names = ['px1', 'py1', 'pz1', 'e1', 'px2', 'py2', 'pz2', 'e2']
    
    # Convert momenta into a NumPy array
    data = np.array([
        [p1.px, p1.py, p1.pz, p1.e, p2.px, p2.py, p2.pz, p2.e]
        for (p1, p2) in momenta
    ])
    
    # Save to NPZ with both data and names
    np.savez(output_path, data=data, names=names)



/tmp/ipykernel_1176167/796966141.py:2: DeprecationWarning: importing ReaderAsciiHepMC2 from pyhepmc is deprecated, please import from pyhepmc.io
  from pyhepmc import GenEvent, GenParticle, ReaderAsciiHepMC2, FourVector


In [ ]:
momenta = extract_electron_momenta_from_hepmc2([
    f"../DATA/HEPMCfiles/HEPMC.43646139._0000{str(i).zfill(2)}.hepmc" for i in range(1, 11)
])

In [ ]:
write_momenta_to_npz(momenta, "../DATA/MomentumOrdered100-000Events")